In [19]:
import sys, os

import pandas as pd

from CART import *
from Utils.plotting import  *
from scipy.stats import norm as ndist
import joblib

# For tree-values
import rpy2.robjects.packages as rpackages
from rpy2.robjects.vectors import StrVector

# Select a CRAN mirror to download from
utils = rpackages.importr('utils')
utils.chooseCRANmirror(ind=1)  # Select the first mirror

# Install 'remotes' if it's not already installed
if not rpackages.isinstalled('remotes'):
    utils.install_packages(StrVector(('remotes',)))

import rpy2.robjects as ro

from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects import numpy2ri

In [20]:
# Run the GitHub installation command for 'treevalues'
ro.r('remotes::install_github("anna-neufeld/treevalues")')
ro.r('library(treevalues)')
ro.r('library(rpart)')

R[write to console]: Using GitHub PAT from the git credential store.

R[write to console]: Skipping install of 'treevalues' from a github remote, the SHA1 (55573782) has not changed since last install.
  Use `force = TRUE` to force installation



In [21]:
def generate_test(mu, sd_y):
    n = mu.shape[0]
    return mu + np.random.normal(size=(n,), scale=sd_y)

In [73]:
class RegressionTree:
    def __init__(self, min_samples_split=2, max_depth=float('inf'),
                 min_proportion=0.2, min_bucket=5):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.root = None
        self.min_proportion = min_proportion
        self.min_bucket = min_bucket
        self.terminal_nodes = []
        self.terminal_parents = []
    def fit(self, X, y, sd=1):
        # sd is std. dev. of randomization
        self.X = X
        self.y = y
        self.n = X.shape[0]
        self.root = self._build_tree(X, y, sd=sd)
        #print("Fit sd:", sd)

    def _build_tree(self, X, y, depth=0, membership=None,
                    prev_branch=None, sd=1.):
        #print("Build tree sd:", sd)
        """
        A recursive private function to build the tree
        by repeatedly splitting
        :param X: the covariates of the previous sub-region
        :param y: the response of the previous sub-region
        :param depth: depth of the previous split
        :param sd: std. dev. of randomization
        :return: a node characterizing this split and fitted value
        """
        num_samples, num_features = X.shape
        if depth == 0:
            membership = np.ones((num_samples,))
        else:
            assert membership is not None

        if prev_branch is None:
            prev_branch = []
            # print("pbc:", prev_branch)

        if num_samples >= max(self.min_samples_split, 2) and depth < self.max_depth:
            best_split = self._get_best_split(X, y, num_features, sd_rand=sd)
            if "feature_index" not in best_split.keys():
                print(best_split)
                print(X)
            feature_idx = best_split["feature_index"]
            threshold = best_split["threshold"]
            pos = best_split["position"]
            left_mbsp = self.X[:, feature_idx] <= threshold
            right_mbsp = self.X[:, feature_idx] > threshold
            left_mbsp = left_mbsp * membership  # n x 1 logical vector
            right_mbsp = right_mbsp * membership  # n x 1 logical vector
            # if best_split["gain"] > 0:
            left_prev_branch = prev_branch.copy()
            left_prev_branch.append([feature_idx, pos, 0])
            right_prev_branch = prev_branch.copy()
            right_prev_branch.append([feature_idx, pos, 1])
            # print(left_prev_branch)
            # print(right_prev_branch)
            left_subtree \
                = self._build_tree(best_split["X_left"],
                                   best_split["y_left"],
                                   depth + 1,
                                   membership=left_mbsp,
                                   prev_branch=left_prev_branch,
                                   sd=sd)
            right_subtree \
                = self._build_tree(best_split["X_right"],
                                   best_split["y_right"],
                                   depth + 1,
                                   membership=right_mbsp,
                                   prev_branch=right_prev_branch,
                                   sd=sd)

            leaf_value = self._calculate_leaf_value(y)
            cur_node = TreeNode(value=leaf_value,
                                feature_index=best_split["feature_index"],
                                threshold=best_split["threshold"],
                                pos=pos,
                                left=left_subtree, right=right_subtree,
                                membership=membership, depth=depth,
                                randomization=best_split["randomization"],
                                prev_branch=prev_branch,
                                sd_rand=sd, terminal=False)
            # Add this parent node to subnodes
            left_subtree.prev_node = cur_node
            right_subtree.prev_node = cur_node
            if left_subtree.terminal and right_subtree.terminal:
                #print(cur_node.threshold)
                self.terminal_parents.append(cur_node)
            return cur_node
        leaf_value = self._calculate_leaf_value(y)
        cur_node = TreeNode(value=leaf_value, membership=membership,
                            sd_rand=sd, depth=depth, terminal=True)
        self.terminal_nodes.append(cur_node)
        return cur_node

    def _get_best_split(self, X, y, num_features, sd_rand=1):
        """
        Input (X, y) of a (potentially sub-)region, return information about
        the best split on this regions
        Assuming no ties in features
        :param X: the (sub-)region's covariates
        :param y: the (sub-)region's response
        :param num_features: dimension of X
        :return: a dictionary containing
                {split_feature_idx, (numerical) splitting_threshold,
                split_position, left_sub_region, right_sub_region,
                gain}
        """
        best_split = {}
        min_loss = float('inf')
        num_sample = X.shape[0]
        randomization = np.zeros((num_sample - 1, num_features))
        min_proportion = self.min_proportion
        # Override min_proportion if min_bucket is set
        if self.min_bucket is not None:
            start = self.min_bucket
            end = num_sample - self.min_bucket - 1
        else:
            start = int(np.floor(num_sample * min_proportion))
            end = num_sample - int(np.ceil(num_sample * min_proportion)) - 1
        #print(start, end)
        #print("Get best split sd:", sd_rand)

        for feature_index in range(num_features):
            feature_values = X[:, feature_index]
            feature_values_sorted = feature_values.copy()
            feature_values_sorted.sort()
            #for i in range(len(feature_values_sorted) - 1):
            for i in range(start, end):
                threshold = feature_values_sorted[i]
                X_left, y_left, X_right, y_right = self._split(X, y, feature_index, threshold)
                if len(X_left) > 0 and len(X_right) > 0:
                    #print("entered 1")
                    if sd_rand != 0:
                        omega = np.random.normal(scale=sd_rand)
                    else:
                        omega = 0
                    randomization[i, feature_index] = omega
                    loss = self._calculate_loss(y_left, y_right, omega)
                    if loss < min_loss:
                        #print("entered 2")
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["position"] = i
                        best_split["X_left"] = X_left
                        best_split["y_left"] = y_left
                        best_split["X_right"] = X_right
                        best_split["y_right"] = y_right
                        best_split["loss"] = loss
                        best_split["randomization"] = randomization
                        # best_split[""]
                        min_loss = loss
        return best_split

    def _split(self, X, y, feature_index, threshold):
        left_mask = X[:, feature_index] <= threshold
        right_mask = X[:, feature_index] > threshold
        return X[left_mask], y[left_mask], X[right_mask], y[right_mask]

    def _calculate_information_gain(self, y, y_left, y_right):
        var_total = np.var(y) * len(y)
        var_left = np.var(y_left) * len(y_left)
        var_right = np.var(y_right) * len(y_right)
        return var_total - (var_left + var_right)

    def _calculate_loss(self, y_left, y_right, randomization):
        n1 = len(y_left)
        n2 = len(y_right)
        n = n1 + n2
        """loss = ((np.var(y_left) * n1 + np.var(y_right) * np.sqrt(n2)) / np.sqrt(n1 + n2)
                + randomization)"""
        loss = ( (- n1 * np.mean(y_left) ** 2 - n2 * np.mean(y_right) ** 2) / np.sqrt(n)
                + randomization)
        # Actually need not divide by n1+n2...
        #print("loss:", loss - randomization)
        #print("randomization:", randomization)
        return loss

    #
    def _calculate_leaf_value(self, y):
        """
        :param y: the response of the previous sub-region
        :return: the mean of the region
        """
        return np.mean(y)

    def predict(self, X):
        """
        :param X: the test dataset
        :return: fitted values
        """
        return np.array([self._predict(sample, self.root) for sample in X])

    def _predict(self, sample, tree):
        """
        Recursively searching the tree for the surrounding region of `sample`
        :param sample: the input covariates
        :param tree: the trained tree
        :return: fitted y value of `sample`
        """
        if tree.terminal:
            return tree.value
        feature_value = sample[tree.feature_index]
        if feature_value <= tree.threshold:
            return self._predict(sample, tree.left)
        else:
            return self._predict(sample, tree.right)

    def _approx_log_reference(self, node, grid, nuisance,
                              contrast, norm_contrast, sd=1, sd_rand=1):
        ## TODO: 0. grid is a grid for eta'Y / (sd * norm_contrast);
        ##          first reconstruct eta'Y and then reconstruct Q
        ## TODO: 1. reconstruct Q from the grid
        ## TODO: 2. Perform Laplace approximation for each grid,
        #           and for each node split
        ## TODO: 3. Add back the constant term omitted in Laplace Approximation
        ## TODO: 4. Return reference measure

        prev_branch = node.prev_branch.copy()
        current_depth = node.depth
        ref_hat = np.zeros_like(grid)

        ## TODO: Move the node according to branch when evaluating integrals
        node = self.root

        # norm = np.linalg.norm(contrast)
        depth = 0

        while depth <= current_depth:
            for g_idx, g in enumerate(grid):
                y_grid = g * sd ** 2 * norm_contrast + nuisance
                # TODO: Account for depth here
                # Subsetting the covariates to this current node
                X = self.X[node.membership.astype(bool)]
                y_g = y_grid[node.membership.astype(bool)]
                y_node = self.y[node.membership.astype(bool)]
                y_left = y_grid[node.left.membership.astype(bool)]
                y_right = y_grid[node.right.membership.astype(bool)]
                y_left_obs = self.y[node.left.membership.astype(bool)]
                y_right_obs = self.y[node.right.membership.astype(bool)]
                optimal_loss = self._calculate_loss(y_left, y_right,
                                                    randomization=0)
                opt_loss_obs = self._calculate_loss(y_left_obs, y_right_obs,
                                                    randomization=0)
                j_opt = node.feature_index  # j^*
                s_opt = node.pos  # s^*
                randomization = node.randomization
                S_total, J_total = randomization.shape
                implied_mean = []
                observed_opt = []



                # TODO: Add a layer to account for depth of the tree
                for j in range(J_total):
                    feature_values = X[:, j]
                    feature_values_sorted = feature_values.copy()
                    feature_values_sorted.sort()
                    for s in range(S_total - 1):
                        if not (j == j_opt and s == s_opt):
                            threshold = feature_values_sorted[s]
                            X_left, y_left, X_right, y_right \
                                = self._split(X, y_g, j, threshold)
                            implied_mean_s_j \
                                = optimal_loss - self._calculate_loss(y_left,
                                                                      y_right,
                                                                      randomization=0)
                            # The split of the actually observed Y
                            X_left_o, y_left_o, X_right_o, y_right_o \
                                = self._split(X, y_node, j, threshold)
                            # print(y_left_o.shape)
                            # print(y_right_o.shape)
                            observed_opt_s_j = (opt_loss_obs -
                                                self._calculate_loss(y_left_o,
                                                                     y_right_o,
                                                                     randomization=0)
                                                + (randomization[s_opt, j_opt] -
                                                   randomization[s, j]))
                            # print("s:", s, "j:", j, "sopt:", s_opt, "jopt:", j_opt)

                            # Record the implied mean
                            # and observed optimization variable
                            implied_mean.append(implied_mean_s_j)
                            observed_opt.append(observed_opt_s_j)

                # The implied mean is given by the optimal loss minus
                # the loss at each split
                implied_mean = np.array(implied_mean)
                observed_opt = np.array(observed_opt)
                # print(observed_opt)
                assert np.max(observed_opt) < 0

                # dimension of the optimization variable
                n_opt = len(implied_mean)
                implied_cov = np.ones((n_opt, n_opt)) + np.eye(n_opt)
                prec = (np.eye(n_opt) - np.ones((n_opt, n_opt))
                        / ((n_opt + 1))) / (sd_rand ** 2)

                # TODO: what is a feasible point?
                # TODO: Need to have access to the observed opt var
                #       where we actually pass in g = eta'Y.
                # print("Implied mean", implied_mean)
                # print("feasible point", observed_opt)
                # print("prec", prec)
                # Approximate the selection probability
                sel_prob, _, _ = solve_barrier_tree_nonneg(Q=implied_mean,
                                                           precision=prec,
                                                           feasible_point=None)
                const_term = (implied_mean).T.dot(prec).dot(implied_mean) / 2
                ref_hat[g_idx] += (- sel_prob - const_term)
                print("conjugate norm:", np.linalg.norm(prec.dot(implied_mean)))

            # Move to the next layer
            if depth < current_depth:
                dir = prev_branch[depth][2]
                if dir == 0:
                    node = node.left  # Depend on where the branch demands
                else:
                    node = node.right
                depth += 1
            else:
                depth += 1  # Exit the loop if targeting depth achieved

        return np.array(ref_hat)

    def _condl_approx_log_reference(self, node, grid, nuisance,
                                    norm_contrast, sd=1, sd_rand=1,
                                    reduced_dim=5, use_CVXPY=True):
        ## TODO: 0. grid is a grid for eta'Y / (sd * ||contrast||_2);
        ##          first reconstruct eta'Y and then reconstruct Q
        ## TODO: 1. reconstruct Q from the grid
        ## TODO: 2. Perform Laplace approximation for each grid,
        #           and for each node split
        ## TODO: 3. Add back the constant term omitted in Laplace Approximation
        ## TODO: 4. Return reference measure
        
        r_is_none = reduced_dim is None

        def k_dim_prec(k, sd_rand):
            prec = (np.eye(k) - np.ones((k, k))
                    / ((k + 1))) / (sd_rand ** 2)
            #print("Precision (k-dim):", prec)
            #print("SD_rand:", sd_rand)
            return prec

        def get_cond_dist(mean, cov, cond_idx, rem_idx, rem_val,
                          sd_rand, rem_dim):
            prec_rem = k_dim_prec(k=rem_dim, sd_rand=sd_rand)

            cond_mean = mean[cond_idx] + cov[np.ix_(cond_idx, rem_idx)].dot(prec_rem).dot(rem_val - mean[rem_idx])
            cond_cov = cov[np.ix_(cond_idx, cond_idx)] - cov[np.ix_(cond_idx, rem_idx)].dot(prec_rem).dot(
                cov[np.ix_(rem_idx, cond_idx)])
            cond_prec = np.linalg.inv(cond_cov)

            return cond_mean, cond_cov, cond_prec

        def get_log_pdf(observed_opt, implied_mean, rem_idx, sd_rand, rem_dim):
            x = observed_opt[rem_idx]
            mean = implied_mean[rem_idx]

            return -1/2 * (np.linalg.norm(x-mean)**2 - np.sum(x-mean)**2/(rem_dim+1)) / sd_rand**2

        prev_branch = node.prev_branch.copy()
        current_depth = node.depth
        ref_hat = np.zeros_like(grid)

        node = self.root

        # norm = np.linalg.norm(contrast)
        depth = 0

        while depth <= current_depth:
            # Subsetting the covariates to this current node
            X = self.X[node.membership.astype(bool)]
            j_opt = node.feature_index  # j^*
            s_opt = node.pos  # s^*
            randomization = node.randomization
            S_total, J_total = randomization.shape

            # Sort feature values to get the threshold
            feature_values_sorted = np.zeros_like(X)
            for j in range(J_total):
                feature_values_sorted[:, j] = X[:, j].copy()
                feature_values_sorted[:, j].sort()

            for g_idx, g in enumerate(grid):
                # norm_contrast: eta / (||eta|| * sigma)
                # grid is a grid for eta'y / (||eta|| * sigma)
                y_grid = g * sd ** 2 * norm_contrast + nuisance

                # Reconstructing y
                y_g = y_grid[node.membership.astype(bool)]
                y_node = self.y[node.membership.astype(bool)]
                y_left = y_grid[node.left.membership.astype(bool)]
                y_right = y_grid[node.right.membership.astype(bool)]
                y_left_obs = self.y[node.left.membership.astype(bool)]
                y_right_obs = self.y[node.right.membership.astype(bool)]
                optimal_loss = self._calculate_loss(y_left, y_right,
                                                    randomization=0)
                opt_loss_obs = self._calculate_loss(y_left_obs, y_right_obs,
                                                    randomization=0)

                implied_mean = []
                observed_opt = []

                # Iterate over all covariates
                for j in range(J_total):
                    num_sample = X.shape[0]
                    min_proportion = self.min_proportion
                    # Override min_proportion if min_bucket is set
                    if self.min_bucket is not None:
                        start = self.min_bucket
                        end = num_sample - self.min_bucket - 1
                    else:
                        start = int(np.floor(num_sample * min_proportion))
                        end = num_sample - int(np.ceil(num_sample * min_proportion)) - 1

                    # for s in range(S_total - 1):
                    for s in range(start, end):
                        if not (j == j_opt and s == s_opt):
                            threshold = feature_values_sorted[s,j]
                            X_left, y_left, X_right, y_right \
                                = self._split(X, y_g, j, threshold)
                            implied_mean_s_j \
                                = optimal_loss - self._calculate_loss(y_left,
                                                                      y_right,
                                                                      randomization=0)
                            # The split of the actually observed Y
                            X_left_o, y_left_o, X_right_o, y_right_o \
                                = self._split(X, y_node, j, threshold)
                            # print(y_left_o.shape)
                            # print(y_right_o.shape)
                            observed_opt_s_j = (opt_loss_obs -
                                                self._calculate_loss(y_left_o,
                                                                     y_right_o,
                                                                     randomization=0)
                                                + (randomization[s_opt, j_opt] -
                                                   randomization[s, j]))
                            # print("s:", s, "j:", j, "sopt:", s_opt, "jopt:", j_opt)

                            # Record the implied mean
                            # and observed optimization variable
                            implied_mean.append(implied_mean_s_j)
                            observed_opt.append(observed_opt_s_j)

                # The implied mean is given by the optimal loss minus
                # the loss at each split
                implied_mean = np.array(implied_mean)
                observed_opt = np.array(observed_opt)
                if np.max(observed_opt) >= 0:
                    print(observed_opt)
                assert np.max(observed_opt) < 0
                
                if r_is_none:
                    reduced_dim = int(len(implied_mean) * 0.3)
                    #print(reduced_dim)
                
                # Get the order of optimization variables in descending order
                obs_opt_order = np.argsort(observed_opt)[::-1]
                #print("obs_opt_order", len(obs_opt_order))
                #print("unique vals", len(np.unique(obs_opt_order)))
                # reduced_dim = max(int(0.1*len(implied_mean)), 5)
                top_d_idx = obs_opt_order[0:reduced_dim]
                rem_d_idx = obs_opt_order[reduced_dim:]
                offset_val = observed_opt[obs_opt_order[reduced_dim]]
                # print("LB:", offset_val)

                linear = np.zeros((reduced_dim * 2, reduced_dim))
                linear[0:reduced_dim, 0:reduced_dim] = np.eye(reduced_dim)
                linear[reduced_dim:, 0:reduced_dim] = -np.eye(reduced_dim)
                offset = np.zeros(reduced_dim * 2)
                offset[reduced_dim:] = -offset_val
                # dimension of the optimization variable
                n_opt = len(implied_mean)
                implied_cov = (np.ones((n_opt, n_opt)) + np.eye(n_opt)) * (sd_rand ** 2)
                cond_implied_mean, cond_implied_cov, cond_implied_prec = (
                    get_cond_dist(mean=implied_mean,
                                  cov=implied_cov,
                                  cond_idx=top_d_idx,
                                  rem_idx=rem_d_idx,
                                  rem_val=observed_opt[rem_d_idx],
                                  sd_rand=sd_rand,
                                  rem_dim=n_opt - reduced_dim))

                if use_CVXPY:
                    ### USE CVXPY
                    # Define the variable
                    o = cp.Variable(reduced_dim)
                    # print(n_opt)
                    # print(len(cond_implied_mean))

                    # Objective function: (1/2) * (u - Q)' * A * (u - Q)
                    objective = cp.Minimize(0.5 * cp.quad_form(o - cond_implied_mean,
                                                               cond_implied_prec))
                    # Constraints: con_linear' * u <= con_offset
                    constraints = [o >= offset_val, o <= 0]
                    # print(offset_val)
                    # Problem definition
                    prob = cp.Problem(objective, constraints)
                    # Solve the problem
                    prob.solve()
                    ref_hat[g_idx] += (-prob.value)
                    # Add omitted term
                    ref_hat[g_idx] += (get_log_pdf(observed_opt=observed_opt,
                                                   implied_mean=implied_mean,
                                                   rem_idx=rem_d_idx,
                                                   sd_rand=sd_rand,
                                                   rem_dim=n_opt - reduced_dim))
                else:
                    sel_prob, _, _ = solve_barrier_tree_box_PGD(Q=cond_implied_mean,
                                                                precision=cond_implied_prec,
                                                                lb=offset_val,
                                                                feasible_point=None)
                    const_term = (cond_implied_mean).T.dot(cond_implied_prec).dot(cond_implied_mean) / 2
                    ref_hat[g_idx] += (- sel_prob - const_term)
                    # Add omitted term
                    ref_hat[g_idx] += (get_log_pdf(observed_opt=observed_opt,
                                                   implied_mean=implied_mean,
                                                   rem_idx=rem_d_idx,
                                                   sd_rand=sd_rand,
                                                   rem_dim=n_opt - reduced_dim))

            # Move to the next layer
            if depth < current_depth:
                dir = prev_branch[depth][2]
                if dir == 0:
                    node = node.left  # Depend on where the branch demands
                else:
                    node = node.right
                depth += 1
            else:
                depth += 1  # Exit the loop if targeting depth achieved

        return np.array(ref_hat)

    def split_inference(self, node, ngrid=1000, ncoarse=20, grid_width=15,
                        sd=1, level=0.9):
        """
        Inference for a split of a node
        :param node: the node whose split is of interest
        :return: p-values for difference in mean
        """
        # First determine the projection direction
        left_membership = node.left.membership
        right_membership = node.right.membership
        contrast = left_membership / np.sum(left_membership) - right_membership / np.sum(right_membership)
        sd_rand = node.sd_rand

        norm_contrast = contrast / (np.linalg.norm(contrast) * sd)

        # Using the normalized contrast in practice
        # for scale-free grid approximation
        observed_target = norm_contrast @ self.y
        # The nuisance parameter is defined the same way
        # as on papers
        nuisance = (self.y - np.linalg.outer(contrast, contrast)
                    @ self.y / (np.linalg.norm(contrast) ** 2))

        stat_grid = np.linspace(-grid_width, grid_width,
                                num=ngrid)

        if ncoarse is not None:
            coarse_grid = np.linspace(-grid_width, grid_width, ncoarse)
            eval_grid = coarse_grid
        else:
            eval_grid = stat_grid

        # Evaluate reference measure (selection prob.) over stat_grid
        ref = self._approx_log_reference(node=node,
                                         grid=eval_grid,
                                         nuisance=nuisance,
                                         contrast=contrast,
                                         norm_contrast=norm_contrast, sd=1,
                                         sd_rand=sd_rand)

        if ncoarse is None:
            logWeights = np.zeros((ngrid,))
            for g in range(ngrid):
                # Evaluate the log pdf as a sum of (log) gaussian pdf
                # and (log) reference measure
                # TODO: Check if the original exp. fam. density is correct
                logWeights[g] = (- 0.5 * (stat_grid[g]) ** 2 + ref[g])
            # normalize logWeights
            logWeights = logWeights - np.max(logWeights)
            condl_density = discrete_family(eval_grid,
                                            np.exp(logWeights),
                                            logweights=logWeights)
        else:
            # print("Coarse grid")
            approx_fn = interp1d(eval_grid,
                                 ref,
                                 kind='quadratic',
                                 bounds_error=False,
                                 fill_value='extrapolate')
            grid = np.linspace(-grid_width, grid_width, num=ngrid)
            sel_probs = np.zeros((ngrid,))
            logWeights = np.zeros((ngrid,))
            for g in range(ngrid):
                # TODO: Check if the original exp. fam. density is correct
                logWeights[g] = (- 0.5 * (grid[g]) ** 2 + approx_fn(grid[g]))
                sel_probs[g] = approx_fn(grid[g])

            # normalize logWeights
            logWeights = logWeights - np.max(logWeights)

            condl_density = discrete_family(grid, np.exp(logWeights),
                                            logweights=logWeights)

        if np.isnan(logWeights).sum() != 0:
            print("logWeights contains nan")
        elif (logWeights == np.inf).sum() != 0:
            print("logWeights contains inf")
        elif (np.asarray(ref) == np.inf).sum() != 0:
            print("ref contains inf")
        elif (np.asarray(ref) == -np.inf).sum() != 0:
            print("ref contains -inf")
        elif np.isnan(np.asarray(ref)).sum() != 0:
            print("ref contains nan")

        """interval = (condl_density.equal_tailed_interval
                        (observed=contrast.T @ self.y,
                         alpha=1-level))
        if np.isnan(interval[0]) or np.isnan(interval[1]):
            print("Failed to construct intervals: nan")"""

        # TODO: Fix this; pass in observed values
        pivot = condl_density.ccdf(x=observed_target
                                     / (np.linalg.norm(contrast) * sd),
                                   theta=0)

        return (pivot, condl_density, contrast, norm_contrast,
                observed_target, logWeights, sel_probs)

    def condl_split_inference(self, node, ngrid=1000, ncoarse=20, grid_w_const=1.5,
                              sd=1, reduced_dim=5, use_cvxpy=False):
        """
        Inference for a split of a node
        :param node: the node whose split is of interest
        :return: p-values for difference in mean
        """
        # First determine the projection direction
        left_membership = node.left.membership
        right_membership = node.right.membership
        contrast = left_membership / np.sum(left_membership) - right_membership / np.sum(right_membership)
        sd_rand = node.sd_rand

        # Normalized contrast: The inner product norm_contrast'Y has sd = 1.
        norm_contrast = contrast / (np.linalg.norm(contrast) * sd)

        # Using the normalized contrast in practice
        # for scale-free grid approximation
        observed_target = norm_contrast @ self.y
        # The nuisance parameter is defined the same way
        # as on papers
        nuisance = (self.y - np.linalg.outer(contrast, contrast)
                    @ self.y / (np.linalg.norm(contrast) ** 2))

        grid_width = grid_w_const * np.abs(observed_target)

        stat_grid = np.linspace(-grid_width, grid_width, num=ngrid)

        if ncoarse is not None:
            coarse_grid = np.linspace(-grid_width, grid_width, ncoarse)
            eval_grid = coarse_grid
        else:
            eval_grid = stat_grid

        ref = self._condl_approx_log_reference(node=node,
                                               grid=eval_grid,
                                               nuisance=nuisance,
                                               norm_contrast=norm_contrast, sd=sd,
                                               sd_rand=sd_rand,
                                               reduced_dim=reduced_dim,
                                               use_CVXPY=use_cvxpy)

        if ncoarse is None:
            logWeights = np.zeros((ngrid,))
            for g in range(ngrid):
                # Evaluate the log pdf as a sum of (log) gaussian pdf
                # and (log) reference measure
                # TODO: Check if the original exp. fam. density is correct
                logWeights[g] = (- 0.5 * (stat_grid[g]) ** 2 + ref[g])
            # normalize logWeights
            logWeights = logWeights - np.max(logWeights)
            condl_density = discrete_family(eval_grid,
                                            np.exp(logWeights),
                                            logweights=logWeights)
        else:
            # print("Coarse grid")
            approx_fn = interp1d(eval_grid,
                                 ref,
                                 kind='quadratic',
                                 bounds_error=False,
                                 fill_value='extrapolate')
            grid = np.linspace(-grid_width, grid_width, num=ngrid)
            logWeights = np.zeros((ngrid,))
            suff = np.zeros((ngrid,))
            sel_probs = np.zeros((ngrid,))
            for g in range(ngrid):
                # TODO: Check if the original exp. fam. density is correct

                logWeights[g] = (- 0.5 * (grid[g]) ** 2 + approx_fn(grid[g]))
                suff[g] = - 0.5 * (grid[g]) ** 2
                sel_probs[g] = approx_fn(grid[g])

            # normalize logWeights
            logWeights = logWeights - np.max(logWeights)

            # condl_density is a discrete approximation
            # to the exponential family distribution with
            # natural parameter theta := eta'mu / (||eta|| * sigma)
            # and
            # sufficient statistic X := eta'y / (||eta|| * sigma) = norm_contrast'Y
            condl_density = discrete_family(grid, np.exp(logWeights),
                                            logweights=logWeights)

        if np.isnan(logWeights).sum() != 0:
            print("logWeights contains nan")
        elif (logWeights == np.inf).sum() != 0:
            print("logWeights contains inf")
        elif (np.asarray(ref) == np.inf).sum() != 0:
            print("ref contains inf")
        elif (np.asarray(ref) == -np.inf).sum() != 0:
            print("ref contains -inf")
        elif np.isnan(np.asarray(ref)).sum() != 0:
            print("ref contains nan")

        """interval = (condl_density.equal_tailed_interval
                        (observed=contrast.T @ self.y,
                         alpha=1-level))
        if np.isnan(interval[0]) or np.isnan(interval[1]):
            print("Failed to construct intervals: nan")"""

        # TODO: Fix this; pass in observed values
        pivot = condl_density.ccdf(x=observed_target,
                                   theta=0)

        """# Recall: observed_target = norm_contrast @ self.y
        L, U = condl_density.equal_tailed_interval(observed=observed_target,
                                                   alpha=0.1)

        print('CI:', L, ',', U)"""

        return (pivot, condl_density, contrast, norm_contrast,
                observed_target, logWeights, suff, sel_probs)

    def condl_node_inference(self, node, ngrid=1000, ncoarse=20, grid_w_const=1.5,
                             sd=1, reduced_dim=5, use_cvxpy=False):
        """
        Inference for a split of a node
        :param node: the node whose split is of interest
        :return: p-values for difference in mean
        """
        # First determine the projection direction
        membership = node.membership
        contrast = membership / np.sum(membership)
        sd_rand = node.sd_rand
        #print("Inference sd", sd_rand)

        # Normalized contrast: The inner product norm_contrast'Y has sd = 1.
        norm_contrast = contrast / (np.linalg.norm(contrast) * sd)

        # Using the normalized contrast in practice
        # for scale-free grid approximation
        observed_target = norm_contrast @ self.y
        # The nuisance parameter is defined the same way
        # as on papers
        nuisance = (self.y - np.linalg.outer(contrast, contrast)
                    @ self.y / (np.linalg.norm(contrast) ** 2))

        grid_width = grid_w_const * np.abs(observed_target)

        stat_grid = np.linspace(-grid_width, grid_width, num=ngrid)

        if ncoarse is not None:
            coarse_grid = np.linspace(-grid_width, grid_width, ncoarse)
            eval_grid = coarse_grid
        else:
            eval_grid = stat_grid

        ref = self._condl_approx_log_reference(node=node.prev_node,
                                               grid=eval_grid,
                                               nuisance=nuisance,
                                               norm_contrast=norm_contrast, sd=sd,
                                               sd_rand=sd_rand,
                                               reduced_dim=reduced_dim,
                                               use_CVXPY=use_cvxpy)

        if ncoarse is None:
            logWeights = np.zeros((ngrid,))
            for g in range(ngrid):
                # Evaluate the log pdf as a sum of (log) gaussian pdf
                # and (log) reference measure
                # TODO: Check if the original exp. fam. density is correct
                logWeights[g] = (- 0.5 * (stat_grid[g]) ** 2 + ref[g])
            # normalize logWeights
            logWeights = logWeights - np.max(logWeights)
            condl_density = discrete_family(eval_grid,
                                            np.exp(logWeights),
                                            logweights=logWeights)
        else:
            # print("Coarse grid")
            approx_fn = interp1d(eval_grid,
                                 ref,
                                 kind='quadratic',
                                 bounds_error=False,
                                 fill_value='extrapolate')
            grid = np.linspace(-grid_width, grid_width, num=ngrid)
            logWeights = np.zeros((ngrid,))
            suff = np.zeros((ngrid,))
            sel_probs = np.zeros((ngrid,))
            for g in range(ngrid):
                # TODO: Check if the original exp. fam. density is correct

                logWeights[g] = (- 0.5 * (grid[g]) ** 2 + approx_fn(grid[g]))
                suff[g] = - 0.5 * (grid[g]) ** 2
                sel_probs[g] = approx_fn(grid[g])

            # normalize logWeights
            logWeights = logWeights - np.max(logWeights)

            # condl_density is a discrete approximation
            # to the exponential family distribution with
            # natural parameter theta := eta'mu / (||eta|| * sigma)
            # and
            # sufficient statistic X := eta'y / (||eta|| * sigma) = norm_contrast'Y
            condl_density = discrete_family(grid, np.exp(logWeights),
                                            logweights=logWeights)

        if np.isnan(logWeights).sum() != 0:
            print("logWeights contains nan")
        elif (logWeights == np.inf).sum() != 0:
            print("logWeights contains inf")
        elif (np.asarray(ref) == np.inf).sum() != 0:
            print("ref contains inf")
        elif (np.asarray(ref) == -np.inf).sum() != 0:
            print("ref contains -inf")
        elif np.isnan(np.asarray(ref)).sum() != 0:
            print("ref contains nan")

        """interval = (condl_density.equal_tailed_interval
                        (observed=contrast.T @ self.y,
                         alpha=1-level))
        if np.isnan(interval[0]) or np.isnan(interval[1]):
            print("Failed to construct intervals: nan")"""

        # TODO: Fix this; pass in observed values
        pivot = condl_density.ccdf(x=observed_target,
                                   theta=0)

        """# Recall: observed_target = norm_contrast @ self.y
        L, U = condl_density.equal_tailed_interval(observed=observed_target,
                                                   alpha=0.1)

        print('CI:', L, ',', U)"""

        return (pivot, condl_density, contrast, norm_contrast,
                observed_target, logWeights, suff, sel_probs)

    def _delete_children(self, node):
        """
        :param node: The node whose children are to be deleted
        :return:
        """
        node.left = None
        node.right = None
        # Keep track of the terminal nodes
        node.terminal = True


    def bottom_up_pruning(self, level=0.1, sd_y=1):
        temp_term_parents = []
        while self.terminal_parents:
            parent = self.terminal_parents.pop()
            pivot, dist, contrast, norm_contrast, obs_tar, logW, suff, sel_probs = (
                self.condl_split_inference(node=parent,
                                           ngrid=10000,
                                           ncoarse=200,
                                           grid_w_const=2.5,
                                           reduced_dim=1,
                                           sd=sd_y,
                                           use_cvxpy=True))

            # Prune if the split is insignificant
            if min(pivot, 1-pivot) >= level/2:
                self._delete_children(parent)
                if parent.prev_branch:
                    if parent.prev_branch[-1][2] == 0:
                        neighbor = parent.prev_node.right
                    else:
                        neighbor = parent.prev_node.left
                    # If this parent node's parent is now a terminal parent node
                    # add it to the terminal parents list
                    if neighbor.terminal:
                        self.terminal_parents.append(parent.prev_node)
            else:
                # If the split is significant,
                # preserve it in the temp list
                temp_term_parents.append(parent)

        self.terminal_parents = temp_term_parents


    def print_branches(self, node=None, start=True, depth=0):
        """
        Recursively printing (with proper indentation denoting depth) the tree
        :param node: the node to be printed
        :param start: a logic flag for whether the node is the root
        :param depth: depth of a node to be printed
        """
        if start:
            node = self.root
        if node is None:
            return
        if node.left or node.right:
            print("\t" * depth, "j:", node.feature_index)
            print("\t" * depth, "threshold:", node.threshold)
            if node.left and node.right:
                print("\t" * depth, "left:")
                self.print_branches(node.left, start=False, depth=depth + 1)
                print("\t" * depth, "right:")
                self.print_branches(node.right, start=False, depth=depth + 1)
            elif node.right:
                print("\t" * depth, "left:")
                self.print_branches(node.right, start=False, depth=depth + 1)
            else:
                print("\t" * depth, "right:")
                self.print_branches(node.left, start=False, depth=depth + 1)
        return

    # Example usage:

# Tree-values inference

In [74]:
def tree_values_inference(X, y, mu, sd_y, max_depth=5, level=0.1,
                          X_test=None):
    # Convert the NumPy matrix to an R matrix
    X_r = numpy2ri.py2rpy(X)
    y_r = numpy2ri.py2rpy(y)

    # Assign the R matrix to a variable in the R environment (optional)
    ro.globalenv['X_r'] = X_r
    ro.globalenv['y_r'] = y_r
    ro.globalenv['p'] = X.shape[1]

    # Construct dataset
    ro.r('data <- cbind(y_r, X_r)')
    # Set the column names to "y", "x1", "x2", ..., "x10"
    ro.r('colnames(data) <- c("y", paste0("x", 1:p))')
    ro.r('data = data.frame(data)')

    # Define the rpart tree model
    tree_cmd = ('bls.tree <- rpart(y ~ ., data=data, model = TRUE, ' +
                'control = rpart.control(cp=0.00, minsplit = 50, minbucket = 20, maxdepth=') + str(max_depth) + '))'
    ro.r(tree_cmd)
    bls_tree = ro.r('bls.tree')
    # Plot the tree values (this will plot directly if you have a plotting backend set up)
    # ro.r('treeval.plot(bls.tree, inferenceType=0)')

    # ro.r('print(row.names(bls.tree$frame)[bls.tree$frame$var == "<leaf>"])')
    ro.r('leaf_idx <- (row.names(bls.tree$frame)[bls.tree$frame$var == "<leaf>"])')
    leaf_idx = ro.r['leaf_idx']

    # Get node mapping
    ro.r('idx_full <- 1:nrow(bls.tree$frame)')
    ro.r('mapped_idx <- idx_full[bls.tree$frame$var == "<leaf>"]')

    len = []
    coverage = []
    len_naive = []
    coverage_naive = []

    for i, idx in enumerate(leaf_idx):
        # Get the branch information for a specific branch in the tree
        command = 'branch <- getBranch(bls.tree, ' + str(idx) + ')'
        ro.r(command)
        # Perform branch inference
        ro.r(f'result <- branchInference(bls.tree, branch, type="reg", alpha = 0.10, sigma_y={sd_y})')
        # Get confidence intervals
        confint = ro.r('result$confint')
        len.append(confint[1] - confint[0])

        target_cmd = "contrast <- (bls.tree$where == mapped_idx[" + str(i + 1) + "])"
        ro.r(target_cmd)
        contrast = ro.r('contrast')
        contrast = np.array(contrast)

        contrast = np.array(contrast * 1 / np.sum(contrast))

        target = contrast.dot(mu)
        coverage.append(target >= confint[0] and target <= confint[1])

        # Naive after tree value
        # Confidence intervals
        naive_CI = [contrast.dot(y) -
                    np.linalg.norm(contrast) * sd_y * ndist.ppf(1 - level / 2),
                    contrast.dot(y) +
                    np.linalg.norm(contrast) * sd_y * ndist.ppf(1 - level / 2)]
        coverage_naive.append((target >= naive_CI[0] and target <= naive_CI[1]))
        len_naive.append(naive_CI[1] - naive_CI[0])

    if X_test is not None:
        X_test_r = numpy2ri.py2rpy(X_test)
        ro.globalenv['X_test_r'] = X_test_r
        ro.r('pred <- predict(bls.tree, data = X_test_r)')
        pred = ro.r['pred']
    else:
        pred = None

    return (np.mean(coverage), np.mean(len),
            np.mean(coverage_naive), np.mean(len_naive), pred)

# RRT inference

In [75]:
def randomized_inference(reg_tree, sd_y, y, mu, level=0.1):
    # print(reg_tree.terminal_nodes)
    coverage_i = []
    lengths_i = []

    for node in reg_tree.terminal_nodes:
        pval, dist, contrast, norm_contrast, obs_tar, logW, suff, sel_probs \
            = (reg_tree.condl_node_inference(node=node,
                                             ngrid=10000,
                                             ncoarse=50,
                                             grid_w_const=5,
                                             reduced_dim=None,
                                             sd=sd_y,
                                             use_cvxpy=True))
        target = contrast.dot(mu)

        # This is an interval for
        # eta_*'mu = eta'mu / (norm(eta) * sd_y)
        selective_CI = (dist.equal_tailed_interval(observed=norm_contrast.dot(y),
                                                   alpha=level))
        selective_CI = np.array(selective_CI)
        selective_CI *= np.linalg.norm(contrast) * sd_y
        coverage_i.append((target >= selective_CI[0] and target <= selective_CI[1]))
        lengths_i.append(selective_CI[1] - selective_CI[0])

    return coverage_i, lengths_i

# Inference with UV decomposition

In [76]:
def UV_decomposition(X, y, mu, sd_y,
                     max_depth=5, min_prop=0, min_sample=10, min_bucket=5,
                     level=0.1, gamma=1,
                     X_test=None):
    n = X.shape[0]
    W = np.random.normal(loc=0, scale=sd_y * np.sqrt(gamma), size=(n,))
    U = y + W
    V = y - W / gamma
    sd_V = sd_y * np.sqrt(1 + 1 / gamma)
    reg_tree = RegressionTree(min_samples_split=min_sample, max_depth=max_depth,
                              min_proportion=min_prop, min_bucket=min_bucket)
    reg_tree.fit(X, U, sd=0)

    coverage = []
    lengths = []

    for node in reg_tree.terminal_nodes:
        contrast = node.membership

        contrast = np.array(contrast * 1 / np.sum(contrast))

        target = contrast.dot(mu)

        # Naive after tree value
        # Confidence intervals
        CI = [contrast.dot(V) -
              np.linalg.norm(contrast) * sd_V * ndist.ppf(1 - level / 2),
              contrast.dot(V) +
              np.linalg.norm(contrast) * sd_V * ndist.ppf(1 - level / 2)]
        coverage.append((target >= CI[0] and target <= CI[1]))
        lengths.append(CI[1] - CI[0])

    if X_test is not None:
        pred = reg_tree.predict(X_test)
    else:
        pred = None

    return coverage, lengths, pred

# Replicating Figure 1

In [77]:
def terminal_inference_sim(n=50, p=5, a=0.1, b=0.1,
                           sd_y=1,
                           noise_sd_list=[0.5, 1, 2, 5],
                           UV_gamma_list=[],
                           use_nonrand=True,
                           start=0, end=100,
                           level=0.1, path=None):
    method_list = [f"RRT_{sd}" for sd in noise_sd_list]
    if use_nonrand:
        method_list += ["Tree val", "Naive"]
    for gamma in UV_gamma_list:
        method_list.append("UV_" + str(gamma))

    coverage_dict = {m: [] for m in method_list}
    length_dict = {m: [] for m in method_list}
    MSE_dict = {m: [] for m in method_list}

    for i in range(start, end):
        print(i, "th simulation")
        np.random.seed(i + 10000)
        X = np.random.normal(size=(n, p))

        mu = b * ((X[:, 0] <= 0) * (1 + a * (X[:, 1] > 0) + (X[:, 2] * X[:, 1] <= 0)))
        y = mu + np.random.normal(size=(n,), scale=sd_y)
        y_test = generate_test(mu, sd_y)
        
        if use_nonrand:
            # Tree value & naive inference & prediction
            (coverage_treeval, avg_len_treeval,
             coverage_treeval_naive, avg_len_treeval_naive,
             pred_test_treeval) = tree_values_inference(X, y, mu, sd_y=sd_y,
                                                        X_test=X, max_depth=3)
            MSE_test_treeval = (np.mean((y_test - pred_test_treeval) ** 2))

            coverage_dict["Tree val"].append(coverage_treeval)
            length_dict["Tree val"].append(avg_len_treeval)
            MSE_dict["Tree val"].append(MSE_test_treeval)
            coverage_dict["Naive"].append(coverage_treeval_naive)
            length_dict["Naive"].append(avg_len_treeval_naive)
            MSE_dict["Naive"].append(MSE_test_treeval)

        for noise_sd in noise_sd_list:
            # Create and train the regression tree
            reg_tree = RegressionTree(min_samples_split=50, max_depth=3,
                                      min_proportion=0., min_bucket=20)

            reg_tree.fit(X, y, sd=noise_sd * sd_y)

            coverage_i, lengths_i = randomized_inference(reg_tree=reg_tree,
                                                         y=y, sd_y=sd_y, mu=mu,
                                                         level=level)
            pred_test = reg_tree.predict(X)
            MSE_test = (np.mean((y_test - pred_test) ** 2))
            # Record results
            coverage_dict[f"RRT_{noise_sd}"].append(np.mean(coverage_i))
            length_dict[f"RRT_{noise_sd}"].append(np.mean(lengths_i))
            MSE_dict[f"RRT_{noise_sd}"].append(MSE_test)

        for gamma in UV_gamma_list:
            gamma_key = "UV_" + str(gamma)
            # UV decomposition
            coverage_UV, len_UV, pred_UV = UV_decomposition(X, y, mu, sd_y, X_test=X,
                                                            min_prop=0., max_depth=3,
                                                            min_sample=50, min_bucket=20,
                                                            gamma=gamma)
            MSE_UV = (np.mean((y_test - pred_UV) ** 2))
            coverage_dict[gamma_key].append(np.mean(coverage_UV))
            length_dict[gamma_key].append(np.mean(len_UV))
            MSE_dict[gamma_key].append(MSE_UV)

        if path is not None:
            joblib.dump([coverage_dict, length_dict, MSE_dict], path, compress=1)

    return coverage_dict, length_dict, MSE_dict

In [78]:
coverage_dict_fig1, length_dict_fig1, MSE_dict_fig1\
    = terminal_inference_sim(n=200, p=5, a=1, b=2,
                             sd_y=2,
                             noise_sd_list=[1, 2.5, 5, 10],
                             UV_gamma_list=[],
                             use_nonrand=True,
                             start=0, end=3,
                             level=0.1, path=None)

0 th simulation


R[write to console]: 



RRuntimeError: 

In [9]:
# Columns: 
# RRT_c: RRT with external randomization N(0, (c*sd_y)^2)
# Tree val: Tree-values
# Naive: naive inference
# Rows: Each row correspond to one round of simulation
pd.DataFrame(coverage_dict_fig1)

In [30]:
# Columns: 
# RRT_c: RRT with external randomization N(0, (c*sd_y)^2)
# Tree val: Tree-values
# Naive: naive inference
# Rows: Each row correspond to one round of simulation
pd.DataFrame(length_dict_fig1)

,RRT_1,RRT_2.5,RRT_5,RRT_10,Tree val,Naive
0,10.355613,4.244548,2.537076,1.704440,25.308622,1.258967
1,11.256963,4.538982,2.740342,2.473626,2.290091,1.166525
2,10.773288,14.469976,2.653465,1.842266,1.871536,1.178633


In [31]:
# Columns: 
# RRT_c: RRT with external randomization N(0, (c*sd_y)^2)
# Tree val: Tree-values
# Naive: naive inference
# Rows: Each row correspond to one round of simulation
pd.DataFrame(MSE_dict_fig1)

,RRT_1,RRT_2.5,RRT_5,RRT_10,Tree val,Naive
0,4.754626,5.385438,6.062667,5.715567,4.774639,4.774639
1,5.007723,4.517161,5.284633,5.598432,4.747049,4.747049
2,5.004523,4.477550,4.936360,5.975434,4.853868,4.853868


# Replicating Figure 2

In [79]:
coverage_dict_fig2, length_dict_fig2, MSE_dict_fig2\
    = terminal_inference_sim(n=200, p=5, a=1, b=2,
                             sd_y=2,
                             noise_sd_list=[1],
                             UV_gamma_list=[0.1, 0.2, 0.3, 0.4, 0.5],
                             use_nonrand=False,
                             start=0, end=3,
                             level=0.1, path=None)

0 th simulation
1 th simulation
2 th simulation


/Users/yilingh/Desktop/PhD/SI_Codes/SI-CART/Utils/discrete_family.py:148: RuntimeWarning: overflow encountered in exp
  self._partition *= np.exp(_largest)


In [80]:
# Columns: 
# RRT_c: RRT with external randomization N(0, (c*sd_y)^2)
# UV_k: UV decomposition with gamma = k
# Rows: Each row correspond to one round of simulation
pd.DataFrame(coverage_dict_fig2)

,RRT_1,UV_0.1,UV_0.2,UV_0.3,UV_0.4,UV_0.5
0,0.857143,1.000000,0.833333,0.857143,1.000000,0.833333
1,0.666667,0.833333,1.000000,1.000000,0.833333,1.000000
2,0.333333,1.000000,1.000000,0.833333,1.000000,1.000000


In [81]:
# Columns: 
# RRT_c: RRT with external randomization N(0, (c*sd_y)^2)
# UV_k: UV decomposition with gamma = k
# Rows: Each row correspond to one round of simulation
pd.DataFrame(length_dict_fig2)

,RRT_1,UV_0.1,UV_0.2,UV_0.3,UV_0.4,UV_0.5
0,3.901546,3.895045,2.847981,2.610729,2.221367,2.004146
1,4.204104,3.881247,2.857391,2.431454,2.153402,2.011266
2,4.843287,3.892569,2.878134,2.449271,2.205467,2.020659


In [72]:
# Columns: 
# RRT_c: RRT with external randomization N(0, (c*sd_y)^2)
# UV_k: UV decomposition with gamma = k
# Rows: Each row correspond to one round of simulation
pd.DataFrame(MSE_dict_fig2)

,RRT_1,UV_0.1,UV_0.2,UV_0.3,UV_0.4,UV_0.5
0,4.754626,5.724732,5.861563,4.959129,5.453345,5.903943
1,5.007723,4.709248,4.827129,4.688048,4.627214,4.768680
2,5.004523,4.553102,4.918423,4.947749,5.058473,5.104218
